In [1]:
import numpy as np
import pandas as pd
import random
import itertools
import pickle

In [2]:
data=pd.read_csv("DMSC.csv")
data.head()

,ID,Movie_Name_EN,Movie_Name_CN,Crawl_Date,Number,Username,Date,Star,Comment,Like
0,0,Avengers Age of Ultron,复仇者联盟2,2017-01-22,1,然潘,2015-05-13,3,连奥创都知道整容要去韩国。,2404
1,1,Avengers Age of Ultron,复仇者联盟2,2017-01-22,2,更深的白色,2015-04-24,2,非常失望，剧本完全敷衍了事，主线剧情没突破大家可以理解，可所有的人物都缺乏动机，正邪之间、...,1231
2,2,Avengers Age of Ultron,复仇者联盟2,2017-01-22,3,有意识的贱民,2015-04-26,2,2015年度最失望作品。以为面面俱到，实则画蛇添足；以为主题深刻，实则老调重弹；以为推陈出...,1052
3,3,Avengers Age of Ultron,复仇者联盟2,2017-01-22,4,不老的李大爷耶,2015-04-23,4,《铁人2》中勾引钢铁侠，《妇联1》中勾引鹰眼，《美队2》中勾引美国队长，在《妇联2》中终于...,1045
4,4,Avengers Age of Ultron,复仇者联盟2,2017-01-22,5,ZephyrO,2015-04-22,2,虽然从头打到尾，但是真的很无聊啊。,723


In [3]:
pos_f='test1w\\pos_review.pkl'
neg_f='test1w\\neg_review.pkl'

posWords = pickle.load(open(pos_f, 'rb'))
negWords = pickle.load(open(neg_f, 'rb'))

posWords = list(itertools.chain(*posWords))  # 把多维数组解链成一维数组
negWords = list(itertools.chain(*negWords))  # 同理

In [4]:
posWords

['可巧',
 '了',
 '当年',
 '北朝鲜',
 '发动',
 '朝鲜战争',
 '大军',
 '压境',
 '南韩',
 '的',
 '最后',
 '防线',
 '就是',
 '在',
 '釜山',
 '所以',
 '韩国',
 '拍',
 '这个',
 '片子',
 '是',
 '寓意',
 '北朝鲜',
 '都',
 '是',
 '僵尸',
 '嘛',
 '；',
 '还有',
 '那个',
 '车厢',
 '的',
 '门',
 '就',
 '像',
 '38',
 '度',
 '纬线',
 '一样',
 '活活',
 '地',
 '将',
 '一对',
 '姐妹',
 '阴阳',
 '相隔',
 'By',
 '看',
 '完',
 '片子',
 '整个',
 '人',
 '都',
 '不好',
 '了',
 '看',
 '谁',
 '都',
 '觉得',
 '像',
 '僵尸',
 '连',
 '对',
 '着',
 '镜子',
 '也',
 '是',
 '怎么',
 '破',
 '即时',
 '先剧',
 '透',
 '了',
 '仍',
 '觉得',
 '釜山',
 '行',
 '好看',
 '但是',
 '因为',
 '知道',
 '结局',
 '所以',
 '不敢',
 '看',
 '完离',
 '结尾',
 '还有',
 '二十分钟',
 '这时候',
 '父女俩',
 '棒球',
 '情侣',
 '孕妇',
 '流浪汉',
 '还',
 '聚在一起',
 '从',
 '悲剧',
 '里',
 '去',
 '感受',
 '希望',
 '和',
 '爱',
 '这样',
 '的',
 '过程',
 '终究',
 '太',
 '残酷',
 '大众',
 '心理学',
 '喜欢',
 '孔侑',
 '几年',
 '没',
 '看到',
 '过',
 '的',
 '好',
 '片子',
 '了',
 '要是',
 '最后',
 '一枪',
 '开',
 '了',
 '就是',
 '神作',
 '觉得',
 '那些',
 '拒绝',
 '开门',
 '的',
 '人',
 '跟',
 '追',
 '他们',
 '的',
 '丧尸',
 '本质',
 '没什么',
 '不',
 '一样',
 '吧',
 '为了',
 '达成',
 '自己',
 '

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

data_combine=pd.concat([negWords,posWords],axis=0)
data_combine['lenth']=data_combine['words'].apply(lambda s:len(s))

plt.figure(figure=(6,4))
sns.displot(data_combine[data_combine['label']==1]['lenth'],label='pos')
sns.displot(data_combine[data_combine['label']==0]['lenth'],label='neg')
plt.legend(['pos','neg'])

In [5]:
print(len(posWords))
print(len(negWords))
X_orig = np.array(posWords+negWords)
Y_orig = np.array([1 for _ in range(193597)] + [0 for _ in range(177127)]) 

193597
177127


In [8]:
# 
vocab_size=25000
import collections
import math
import os
import random
import zipfile
import numpy as np
import urllib
import tensorflow as tf
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
def build_dataset(words):
    count = [['UNK', -1]]    
    # 词汇频数统计
    count.extend(collections.Counter(words).most_common(vocab_size-1))
    dictionary = dict()    
    # 存入字典
    for word, _ in count:
        dictionary[word] = len(dictionary)

    data = list()
    unk_count = 0 
    # 遍历单词列表判断是否放入字典
    for word in words:        
        if word in dictionary:
            index = dictionary[word]        
        else:
            index = 0
            unk_count += 1
        # 指定单词的频数编码
        data.append(index)
    count[0][1] = unk_count    
    # 反转字典
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))    
    return data, count, dictionary, reverse_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(X_orig)

In [7]:
# 预处理
from keras.preprocessing.text import text_to_word_sequence,one_hot,Tokenizer
from keras.preprocessing.sequence import pad_sequences
import time
vocab_size = 20000
maxlen = 200
print("Start fitting the corpus......")
t = Tokenizer(vocab_size) # 要使得文本向量化时省略掉低频词，就要设置这个参数
tik = time.time()
t.fit_on_texts(X_orig) # 在所有的评论数据集上训练，得到统计信息
tok = time.time()
word_index = t.word_index # 不受vocab_size的影响
print('all_vocab_size',len(word_index))
print("Fitting time: ",(tok-tik),'s')
print("Start vectorizing the sentences.......")
v_X = t.texts_to_sequences(X_orig) # 受vocab_size的影响
print("Start padding......")
pad_X = pad_sequences(v_X,maxlen=maxlen,padding='post')# 对齐
print("Finished!")

Start fitting the corpus......
all_vocab_size 29393
Fitting time:  10.2372465133667 s
Start vectorizing the sentences.......
Start padding......
Finished!


In [8]:
import copy
x = list(t.word_counts.items())
s = sorted(x,key=lambda p:p[1],reverse=True) # 递减
small_word_index = copy.deepcopy(word_index) # 防止原来的字典也被改变了
print("Removing less freq words from word-index dict...")
for item in s[20000:]:
    small_word_index.pop(item[0]) # 除去最低频的词？？
print("Finished!")
print(len(small_word_index))
print(len(word_index))

Removing less freq words from word-index dict...
Finished!
20000
29393


In [9]:
small_word_index

{'的': 1,
 '了': 2,
 '我': 3,
 '是': 4,
 '看': 5,
 '电影': 6,
 '都': 7,
 '很': 8,
 '不': 9,
 '也': 10,
 '就': 11,
 '有': 12,
 '和': 13,
 '在': 14,
 '好': 15,
 '给': 16,
 '人': 17,
 '剧情': 18,
 '你': 19,
 '还': 20,
 '没有': 21,
 '还是': 22,
 '啊': 23,
 '这': 24,
 '说': 25,
 '没': 26,
 '就是': 27,
 '一个': 28,
 '吧': 29,
 '真的': 30,
 '好看': 31,
 '觉得': 32,
 '什么': 33,
 '不错': 34,
 '去': 35,
 '让': 36,
 '喜欢': 37,
 '但': 38,
 '太': 39,
 '特效': 40,
 '多': 41,
 '想': 42,
 '故事': 43,
 '但是': 44,
 '能': 45,
 '感觉': 46,
 '笑': 47,
 '可以': 48,
 '一部': 49,
 '不是': 50,
 '最后': 51,
 '到': 52,
 '一星': 53,
 '被': 54,
 '自己': 55,
 '得': 56,
 '要': 57,
 '又': 58,
 '对': 59,
 '这部': 60,
 '上': 61,
 '完': 62,
 '这么': 63,
 '片子': 64,
 '他': 65,
 '这个': 66,
 '有点': 67,
 '画面': 68,
 '挺': 69,
 '那么': 70,
 '会': 71,
 '小': 72,
 '吗': 73,
 '演技': 74,
 '看到': 75,
 '着': 76,
 '知道': 77,
 '再': 78,
 '比': 79,
 '拍': 80,
 '看过': 81,
 '最': 82,
 '时候': 83,
 '中国': 84,
 '打': 85,
 '“': 86,
 '里': 87,
 '虽然': 88,
 '我们': 89,
 '点': 90,
 '导演': 91,
 '这样': 92,
 '”': 93,
 '个': 94,
 '烂片': 95,
 '把': 96,
 '还有': 97

In [10]:
import gensim
model_file = gensim.models.Word2Vec(small_word_index, size=200, sg=1, iter=8)  
model_file.wv.save_word2vec_format("./word2Vec" + ".bin", binary=True) 

# 加载bin格式的模型
wv_model = gensim.models.KeyedVectors.load_word2vec_format("word2Vec.bin", binary=True)

D:\Anaconda\lib\site-packages\smart_open\smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [11]:
n=0
embedding_matrix = np.random.uniform(size=(vocab_size+1,300)) # +1是要留一个给index=0
print("Transfering to the embedding matrix......")
for word,index in small_word_index.items():
    try:
        word_vector = wv_model[word]
        embedding_matrix[index] = word_vector
    except:
#         print("Word: [",word,"] not in wvmodel! Use random embedding instead.")
        n+=1
print(n)
print("Finished!")
print("Embedding matrix shape:\n",embedding_matrix.shape)

Transfering to the embedding matrix......
20000
Finished!
Embedding matrix shape:
 (20001, 300)


In [12]:
from sklearn.model_selection import train_test_split
np.random.seed = 1
random_indexs = np.random.permutation(len(pad_X))
X = pad_X[random_indexs]
Y = Y_orig[random_indexs]
print(Y[:50])
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.25)
print("X_train:",X_train.shape)
print("y_train:",y_train.shape)
print("X_test:",X_test.shape)
print("y_test:",y_test.shape)
print(list(y_train).count(1))
print(list(y_train).count(0))

[1 1 0 1 1 0 0 1 1 0 1 1 1 0 1 0 1 0 1 1 1 0 0 1 1 1 1 1 0 1 0 1 0 1 0 1 1
 0 0 1 0 0 1 0 1 1 1 0 0 0]
X_train: (278043, 200)
y_train: (278043,)
X_test: (92681, 200)
y_test: (92681,)
145205
132838


In [13]:
from keras import backend as K
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [14]:
# create the model
import keras
from keras.models import Sequential,Model
from keras.layers import Input,Dense,GRU,LSTM,Activation,Dropout,Embedding
from keras.layers import Multiply,Concatenate,Dot
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
wv_dim=300
inputs = Input(shape=(maxlen,))
use_pretrained_wv = True
if use_pretrained_wv:
    wv = Embedding(vocab_size+1,wv_dim,input_length=maxlen,weights=[embedding_matrix])      (inputs)
else:
    wv = Embedding(vocab_size+1,wv_dim,input_length=maxlen)(inputs)

conv=Conv1D(filters=32, kernel_size=3, padding='same', activation='relu')(wv)
pool=MaxPooling1D(pool_size=2)(conv)

h = LSTM(128)(pool)
y = Dense(1,activation='sigmoid')(h)
m = Model(input=inputs,output=y)
m.summary()

m.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy',f1])
m.fit(X_train,y_train,batch_size=256,epochs=6,validation_split=0.15)

scores = m.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Instructions for updating:
Colocations handled automatically by placer.


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 200)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 200, 300)          6000300   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 200, 32)           28832     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 100, 32)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               82432     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 6,111,693
Trainable params: 6,111,693
Non-trainable params: 0
_________________________________________________________________


In [17]:
from keras.layers import Bidirectional
from Attention_layer import Attention_layer

In [ ]:
# 2.词向量+BiLSTM
conv=Conv1D(filters=32, kernel_size=3, padding='same', activation='relu')(wv)
pool=MaxPooling1D(pool_size=2)(conv)

l_gru = Bidirectional(LSTM(128, return_sequences=False))(pool)
y = Dense(1,activation='sigmoid')(l_gru)
m = Model(input=inputs,output=y)
m.summary()

m.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy',f1])
m.fit(X_train,y_train,batch_size=256,epochs=6,validation_split=0.15)

scores = m.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  import sys


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 200)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 200, 300)          6000300   
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 200, 32)           28832     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 100, 32)           0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 256)               164864    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 6,194,253
Trainable params: 6,194,253
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# 3.词向量+BiLSTM+Attention
conv=Conv1D(filters=32, kernel_size=3, padding='same', activation='relu')(wv)
pool=MaxPooling1D(pool_size=2)(conv)

l_gru = Bidirectional(LSTM(128, return_sequences=False))(pool)
l_att = Attention_layer()(l_gru)
y = Dense(1,activation='sigmoid')(l_att)
m = Model(input=inputs,output=y)
m.summary()

m.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy',f1])
m.fit(X_train,y_train,batch_size=256,epochs=6,validation_split=0.15)

scores = m.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [17]:
# model_word = Word2Vec(size=voc_dim,
#                      min_count=min_out,
#                      window=window_size,
#                      workers=cpu_count,
#                      iter=5)
# model_word.build_vocab(X_org)
# model_word.train(X_org, total_examples=model_word.corpus_count, epochs=model_word.iter)
# input_dim = len(model_word.wv.vocab.keys()) + 1 #下标0空出来给不够10的字
# embedding_weights = np.zeros((input_dim, voc_dim)) 
# w2dic={}
# for i in range(len(model_word.wv.vocab.keys())):
#     embedding_weights[i+1, :] = model_word [model_word.wv.vocab.keys()[i]]
#     w2dic[model_word.wv.vocab.keys()[i]]=i+1

Transfering to the embedding matrix......
20000
Finished!
Embedding matrix shape:
 (20001, 300)


In [7]:
import jieba
 
data['text'] = data.comment.apply(lambda x: " ".join(jieba.cut(x)))

ModuleNotFoundError: No module named 'jieba'

In [8]:
import os
sub_path='E:\\Github\\SentimentAnalysis_CNN_RNN'
f=open(os.path.join(sub_path,'DMSC.csv'),encoding='utf-8')
all_lines=f.readlines()
valid=random.sample(all_lines,234)
all_lines=set(all_lines)
valid=set(valid)
train=all_lines-valid
with open(os.path.join(sub_path,'cn_valid.csv'),'a',encoding='utf-8') as valid_csv:
    for valid_name in valid:
        valid_csv.write(valid_name)
with open(os.path.join(sub_path,'cn_train.csv'),'a',encoding='utf-8') as train_csv:
    for train_name in train:
        train_csv.write(train_name)